# 本文主题：镜像的构建

# 1.概述
    镜像的构建主要有两种方法：自动化构建和手动构建，分别对应Docker的 build 和 commit 指令。

## 2.手动构建
    手动构建的原理：从一个现有容器创建一个新的镜像，容器的存储层会保存到镜像中去。
    使用Docker的commit命令来手动构建镜像，但这种方法不如使用Dockerfile自动化构建灵活和可维护。通常建议优先使用Dockerfile来构建镜像。
    使用方法参见课本：P99，可以按照课本案例运行一次。
    这里以Nginx为例子，修改原始Nginx中的主页内容。
    
### 镜像下载
    这是一个基于alpine操作系统的nginx版本
```shell
docker pull nginx:alpine
# 指令输出内容如下
alpine: Pulling from library/nginx
619be1103602: Already exists
ed3e62e73b33: Pull complete
5126dce06df7: Pull complete
1d0dd2dc2265: Pull complete
2b1ab92f0231: Pull complete
6eba808ac059: Pull complete
57038e85fbb8: Pull complete
eec94c9845c0: Pull complete
Digest: sha256:31bad00311cb5eeb8a6648beadcf67277a175da89989f14727420a80e2e76742
Status: Downloaded newer image for nginx:alpine
docker.io/library/nginx:alpine

What's Next?
  View a summary of image vulnerabilities and recommendations → docker scout quickview nginx:alpine

```
    
### 镜像运行
    
```shell
# 浏览镜像
docker images
REPOSITORY   TAG             IMAGE ID       CREATED         SIZE
nginx        alpine          e289a478ace0   8 weeks ago     42.6MB

# 运行
docker run --name my-nginx -p 80:80 -d nginx:alpine
7d5f633169885471ab469f6991ff3d945bc9b8b02828277d4a47848e2599edac

# 查看本机端口号 80 是否启动
netstat -ano | find ":80"
  TCP    0.0.0.0:80             0.0.0.0:0              LISTENING       16108
  TCP    127.0.0.1:6197         127.0.0.1:8073         ESTABLISHED     17176
  TCP    127.0.0.1:8073         127.0.0.1:6197         ESTABLISHED     17176
  TCP    172.28.17.130:6184     172.28.17.200:8000     ESTABLISHED     7812
  TCP    172.28.17.130:6324     43.135.106.161:8080    ESTABLISHED     14620
  TCP    [::]:80                [::]:0                 LISTENING       16108
  TCP    [::1]:80               [::]:0                 LISTENING       3568
  TCP    [240e:399:1:4a10:ec9c:3016:3fa3:806e]:6431  [240e:ff:f100:41::12]:8080  ESTABLISHED     8904

# 使用curl工具，或者通过浏览器访问：http://localhost
where curl
C:\Windows\System32\curl.exe
C:\Users\admin\anaconda3\Library\bin\curl.exe

# curl输出了网页的内容
curl localhost
<!DOCTYPE html>
<html>
<head>
<title>Welcome to nginx!</title>
<style>
html { color-scheme: light dark; }
body { width: 35em; margin: 0 auto;
font-family: Tahoma, Verdana, Arial, sans-serif; }
</style>
</head>
<body>
<h1>Welcome to nginx!</h1>
<p>If you see this page, the nginx web server is successfully installed and
working. Further configuration is required.</p>

<p>For online documentation and support please refer to
<a href="http://nginx.org/">nginx.org</a>.<br/>
Commercial support is available at
<a href="http://nginx.com/">nginx.com</a>.</p>

<p><em>Thank you for using nginx.</em></p>
</body>
</html>
```

### 修改容器内容

```shell
# 进入容器内部
docker exec -it my-nginx ash

# nginx默认目录下的index.html
cd /usr/share/nginx/html
ls
50x.html    index.html
cat index.html
......
# 修改 index.html 网页文件内容
cat <<EOF > /usr/share/nginx/html/index.html
<meta HTTP-EQUIV="Content-Type" CONTENT="text/html; charset=utf8">
<pre>
红泪偷垂，满眼春风百事非。
    -- 纳兰性德《采桑子·当时错》
</pre>
EOF

cat index.html
......

# 退出容器
exit

# 测试，或者通过浏览器访问：http://localhost
curl localhost
<meta HTTP-EQUIV="Content-Type" CONTENT="text/html; charset=utf8">
<pre>
红泪偷垂，满眼春风百事非。
    -- 纳兰性德《采桑子·当时错》
</pre>
```

### 创建新的镜像
    我们希望制作新的镜像：一个诗人v1.0。
```shell
# 创建新的镜像
docker commit --author "a poet" --message "make a poetry" my-nginx poet:1.0
sha256:c21f82d7bb4b40a3942e4f31e58b37c1e99f05cb748b5e55e14c0d0f2ac718c4

# 查询镜像列表
docker images
REPOSITORY         TAG             IMAGE ID       CREATED         SIZE
poet               1.0             c21f82d7bb4b   4 seconds ago   42.6MB
nginx              alpine          e289a478ace0   8 weeks ago     42.6MB
```

### 运行新镜像

```shell
# 删除旧容器
docker stop my-nginx
my-nginx
docker rm my-nginx
my-nginx

# 删除旧镜像
docker rmi nginx:alpine
Untagged: nginx:alpine
Untagged: nginx@sha256:31bad00311cb5eeb8a6648beadcf67277a175da89989f14727420a80e2e76742

# 查询一下容器实例和镜像列表
docker ps -a
CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES

docker images
REPOSITORY         TAG             IMAGE ID       CREATED         SIZE
poet               1.0             c21f82d7bb4b   7 minutes ago   42.6MB

# 使用新镜像运行容器
docker run --name my-poet -p 80:80 -d poet:1.0
235f9b0f72d8ef92b76f49d3c317caddca1b8db7c7a323a9639443beaf45060c

# 测试验证
curl localhost
<meta HTTP-EQUIV="Content-Type" CONTENT="text/html; charset=utf8">
<pre>
红泪偷垂，满眼春风百事非。
    -- 纳兰性德《采桑子·当时错》
</pre>
```
### 根据上节课内容，赶紧将你生成的镜像 push 到私有镜像仓库吧！！！

## 3.自动构建
    手动构建的过程并不优雅，不过它容易上手。特殊情况下也可以解决我们一些棘手问题。不过更为经常使用的是自动构建方法。主要是需要编辑代码文件：“Dockerfile”和使用“docker build”指令。
    
### Dockerfile

    创建一个目录 poet 用于存放我们的代码文件
    创建一个文本文件：“Dockerfile”，注意没有扩展名，其内容如下：
```shell
FROM nginx:alpine
MAINTAINER a poet

RUN cat <<EOF > /usr/share/nginx/html/index.html
<meta HTTP-EQUIV="Content-Type" CONTENT="text/html; charset=utf8">
<pre>
红泪偷垂，满眼春风百事非。
    -- 纳兰性德《采桑子·当时错》
</pre>
EOF
```

### build指令

```shell
# 你需要进入 Dockerfile 文件所在的目录
cd poet
docker  build --tag poet:2.0 .

[+] Building 2.9s (6/6) FINISHED                                                                         docker:default
 => [internal] load build definition from Dockerfile                                                               0.0s
 => => transferring dockerfile: 294B                                                                               0.0s
 => [internal] load metadata for docker.io/library/nginx:alpine                                                    2.1s
 => [internal] load .dockerignore                                                                                  0.1s
 => => transferring context: 2B                                                                                    0.0s
 => [1/2] FROM docker.io/library/nginx:alpine@sha256:31bad00311cb5eeb8a6648beadcf67277a175da89989f14727420a80e2e7  0.3s
 => => resolve docker.io/library/nginx:alpine@sha256:31bad00311cb5eeb8a6648beadcf67277a175da89989f14727420a80e2e7  0.1s
 => [2/2] RUN cat <<EOF > /usr/share/nginx/html/index.html                                                         0.3s
 => exporting to image                                                                                             0.0s
 => => exporting layers                                                                                            0.0s
 => => writing image sha256:dac8c2fa7aff3583956eea0bd73f1f27249b3eaa4b75df6e4c27b1e1f5a134db                       0.0s
 => => naming to docker.io/library/poet:2.0                                                                        0.0s

What's Next?
  View a summary of image vulnerabilities and recommendations → docker scout quickview
```

### 测试验证

```shell
docker images
REPOSITORY         TAG             IMAGE ID       CREATED          SIZE
poet               2.0             dac8c2fa7aff   53 seconds ago   42.6MB
poet               1.0             c21f82d7bb4b   24 minutes ago   42.6MB
```

## 4.构建命令
    请参考课本：P88-P96 关于Dockerfile的主要指令，在未来学习中你可能会经常需要参考这个部分内容。